<a href="https://colab.research.google.com/github/rebeccaastaix/BASC0033/blob/main/gender_assignemnt_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gender-guesser

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 2-12-MAIN-Curated-dataset-(4045-records)-xlsx-3.xlsx to 2-12-MAIN-Curated-dataset-(4045-records)-xlsx-3 (1).xlsx


In [ ]:
import pandas as pd
import re
import numpy as np

titles = {'mr', 'mrs', 'ms', 'miss', 'sir', 'dr', 'prof', 'madame', 'mme',
          'm.', 'lord', 'lady', 'baron', 'baronne', 'count', 'conte', 'don', 'dame'}
particles = {'de', 'du', 'von', 'van', 'da', 'del', 'della', 'la', 'le', 'des'}

def extract_first_name(name):
    """
    Try to extract a usable first name from the 'Name' column.
    Returns a string like 'Alfred' or None if we can't find one.
    """
    if not isinstance(name, str):
        return None

    s = name.strip()
    if not s:
        return None

    # Detection of first name in parentheses
    m = re.search(r'\(([^)]+)\)', s)
    if m:
        inside = m.group(1)
        tokens = re.split(r'[,\s]+', inside)
        for t in tokens:
            t0 = t.strip()
            t_clean = t0.strip(".").lower()
            if (not t_clean or t_clean in titles or t_clean in particles
                or len(t_clean) <= 1 or not t0[0].isupper()):
                continue
            return t_clean.capitalize()

    # Pattern recognition in punctuation
    had_comma = ',' in s
    if had_comma:
        after = s.split(',', 1)[1]  # take part after first comma
        tokens = re.split(r'[,\s]+', after.replace('-', ' '))
        for t in tokens:
            t0 = t.strip()
            t_clean = t0.strip(".").lower()
            if (not t_clean or t_clean in titles or t_clean in particles
                or len(t_clean) <= 1 or t0.endswith('.') or len(t0) == 1
                or not t0[0].isupper()):
                continue
            return t_clean.capitalize()
        return None

    # Otherwise assume "First Last" and take first capitalised word
    tokens = re.split(r'\s+', s.replace('-', ' '))
    for t in tokens:
        t0 = t.strip()
        t_clean = t0.strip(".,").lower()
        if (not t_clean or t_clean in titles or t_clean in particles
            or len(t_clean) <= 1 or not t0[0].isupper()):
            continue
        return t_clean.capitalize()

    return None


In [ ]:
import pandas as pd
df = pd.read_excel('2-12-MAIN-Curated-dataset-(4045-records)-xlsx-3 (1).xlsx')
df['FirstName'] = df['Name'].apply(extract_first_name)
df[['Name', 'FirstName']].head(1000)

,Name,FirstName
0,"Tennyson, Alfred Tennyson, Baron",Alfred
1,"Pope, Alexander",Alexander
2,"Jerningham, mr (Edward)",Edward
3,"Bouilly, Jean Nicolas",Jean
4,NaN,None
...,...,...
995,"Josse, Hector",Hector
996,"Joubert, André",André
997,"Jullien, Adolphe",Adolphe
998,"Kanitz, Felix Philipp",Felix


In [ ]:
import unicodedata

# Removing accents
def remove_accents(txt):
    if not isinstance(txt, str):
        return txt
    # Normalising
    return ''.join(
        c for c in unicodedata.normalize('NFD', txt)
        if unicodedata.category(c) != 'Mn'
    )

# Detecting gender
!pip install gender-guesser
from gender_guesser.detector import Detector
detector = Detector()

# Manual fixes
french_overrides = {
    'jerome': 'Male',
    'edouard': 'Male',
    'emile': 'Male',
    'etienne': 'Male',
    'eustace': 'Male',
    'hercule': 'Male',
    'malthe': 'Male',
    'camille': 'Male',
}

def guess_gender(first):
    if first is None or (isinstance(first, float) and pd.isna(first)):
        return 'Unknown'

    clean = remove_accents(str(first)).strip()
    clean_lower = clean.lower()

    if clean_lower in french_overrides:
        return french_overrides[clean_lower]

    g = detector.get_gender(clean)
    mapping = {
        'male': 'Male',
        'mostly_male': 'Male',
        'female': 'Female',
        'mostly_female': 'Female',
        'andy': 'Unknown',     # androgynous: keep as Unknown
        'unknown': 'Unknown'
    }
    return mapping.get(g, 'Unknown')

df['Gender'] = df['FirstName'].apply(guess_gender)

# 1. "organisation" = Unknown
df.loc[df['Type of name'].str.lower() == 'organisation', 'Gender'] = 'Unknown'

# 2. "Camille" = Male
df.loc[df['FirstName'].str.lower() == 'camille', 'Gender'] = 'Male'


In [ ]:
df['Gender'].value_counts()
df[df['Gender'] == 'Unknown'][['Name', 'FirstName']].head(30)

,Name,FirstName
4,NaN,None
5,"Cossé, P., satirist",None
6,"Driver, F. W.",None
8,NaN,None
10,"Reclus, Elisée",Elisée
11,NaN,None
12,"Join-Lambert, A.",None
16,"Rose, A. R.",None
17,"Chéon, de",None
18,NaN,None


In [ ]:

df.loc[df['Type of name'].str.lower() == 'organisation', 'Gender'] = 'Unknown'

df.loc[df['FirstName'].str.lower() == 'camille', 'Gender'] = 'Male'


In [ ]:
df.to_excel("dataset_with_gender.xlsx", index=False)

In [ ]:
from google.colab import files
df.to_excel("dataset_with_gender.xlsx", index=False)
files.download("dataset_with_gender.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>